In [1]:
import pandas as pd
import numpy as np

# Import warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Upload heart.csv to dataFrame
df_heart = pd.read_csv('heart_disease.csv')

# Show first five rows
df_heart.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


- 'target' is binary, with 1 indicating that the patient has heart disease and 0 indicating that they do not. 
<br>
- age: Age in years <br>
- sex: Sex (1 = male; 0 = female) 
- cp: Chest pain type (1 = typical angina, 2 = atypical angina, 3 = non-anginal pain, 4 = asymptomatic) 
- trestbps: Resting blood pressure (in mm Hg on admission to the hospital) 
- chol: Serum cholesterol in mg/dl 6 
- fbs: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
- restecg: Resting electrocardiographic results (0 = normal, 1 = having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), 2 = showing probable or definite left ventricular hypertrophy by Estes' criteria) 
- thalach: Maximum heart rate achieved 
- exang: Exercise induced angina (1 = yes; 0 = no) 
- oldpeak: ST depression induced by exercise relative to rest 
- slope: The slope of the peak exercise ST segment (1 = upsloping, 2 = flat, 3 = downsloping) 
- ca: Number of major vessels (0-3) colored by fluoroscopy 
- thal: 3 = normal; 6 = fixed defect; 7 = reversible defect<br><br>
<p>
refer to "https://archive.ics.uci.edu/ml/datasets/Heart+Disease"</p>

In [5]:
# split data into X and y
X = df_heart.iloc[:,:-1]
y = df_heart.iloc[:,-1]

In [6]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [10]:
# Import Decision Tree Regressor
from sklearn.tree import DecisionTreeClassifier

# Import cross_val_score
from sklearn.model_selection import cross_val_score

In [11]:
# Initialize Decision Tree Classifier
model = DecisionTreeClassifier()

# Obtain scores of cross-validation
scores = cross_val_score(model, X, y, cv=5)

# Display accuracy
print('Accuracy:', np.round(scores, 2))

# Display mean accuracy
print('Accuracy mean: %0.2f' % (scores.mean()))

Accuracy: [0.74 0.85 0.72 0.68 0.72]
Accuracy mean: 0.74


In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

def grid_search_clf(params, runs=20, clf=DecisionTreeClassifier()):

    # Instantiate GridSearchCV as grid_reg
    grid_clf = GridSearchCV(clf, params, cv=5, n_jobs=-1)
    
    # Fit grid_reg on X_train and y_train
    grid_clf.fit(X_train, y_train)
    
    print(grid_clf.best_params_)

    # Extract best estimator
    best_model = grid_clf.best_estimator_
    
    # Extract best score
    best_score = grid_clf.best_score_

    # Print best score
    print("Training score: {:.3f}".format(best_score))

    # Predict test set labels
    y_pred = best_model.predict(X_test)

    # Compute accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Print accuracy
    print('Test score: {:.3f}'.format(accuracy))
        
    # Return best model
    return best_model

In [19]:
grid_search_clf(params={'criterion':['entropy', 'gini'],
                          'min_samples_split':[2, 3, 4, 5, 6, 8, 10],
                          'min_samples_leaf':[0.01, 0.02, 0.03, 0.04, 0.05],
                          'max_leaf_nodes':[10, 15, 20, 25, 30, 35, 40, 45, 50],
                          'max_depth':[2,4,6,8],
                         })

{'criterion': 'entropy', 'max_depth': 6, 'max_leaf_nodes': 30, 'min_samples_leaf': 0.01, 'min_samples_split': 3}
Training score: 0.806
Test score: 0.820


DecisionTreeClassifier(criterion='entropy', max_depth=6, max_leaf_nodes=30,
                       min_samples_leaf=0.01, min_samples_split=3)

In [20]:
best_clf = DecisionTreeClassifier(criterion='entropy', max_depth=6,
                       max_leaf_nodes=30,
                       min_samples_leaf=0.01, min_samples_split=3)
best_clf.fit(X, y)

DecisionTreeClassifier(criterion='entropy', max_depth=6, max_leaf_nodes=30,
                       min_samples_leaf=0.01, min_samples_split=3)

# Feature importance

In [21]:
best_clf.feature_importances_

array([0.14966508, 0.05286893, 0.26185225, 0.        , 0.07123252,
       0.        , 0.0222739 , 0.05522678, 0.02610328, 0.08897344,
       0.01322719, 0.15819765, 0.10037897])

In [24]:
# Zip columns and feature_importances_ into dict
feature_dict = dict(zip(X.columns, best_clf.feature_importances_))

[(k,v) for k, v in sorted(feature_dict.items(), key=lambda item: item[1], reverse=True)]

[('cp', 0.26185224946865493),
 ('ca', 0.15819765310978204),
 ('age', 0.14966508401442472),
 ('thal', 0.10037897388778953),
 ('oldpeak', 0.08897343617255696),
 ('chol', 0.07123252302264449),
 ('thalach', 0.05522677974022177),
 ('sex', 0.05286892918178917),
 ('exang', 0.026103279671184323),
 ('restecg', 0.02227390105567912),
 ('slope', 0.01322719067527286),
 ('trestbps', 0.0),
 ('fbs', 0.0)]